![Sponsors](img/sponsors.png)

# Computational Design of  Protein Biosensors

**BPDMC**, Wed 07.09.2019

*Per Jr. Greisen*


* Twitter
<i class="icon-twitter icon-2x"><strong> [@GreisenPer](https://twitter.com/GreisenPer)</strong></i>
* github
<i class="icon-github icon-2x"><strong> [pgreisen](https://github.com/pgreisen)</strong></i>
* gmail
<i class="icon-github icon-2x"><strong> [pgreisen@gmail.com](https://github.com/pgreisen)</strong></i>

## About me

A brief bio:

* Director of computational biology @enevolv
    * Synthetic biology company - design of pathways
* Principal scientist at Novo Nordisk
    * Design of antibodies and small scaffolds
* Post doc in Bakerlab
    * Design of small molecule binder
    * Repurposing of enzymes
* Ph.D in physics

# Outline

* Introduction to computational design of biosensors
* Motivation to design proteins as biosensors
* Computational design of biosensors 
    * E.g. Fentanyl
* Specificity of biosensors
    * Off-target effects for small molecules e.g. vitamin D3 
* Applications of biosensors
    * Example of biosensor in yeast for metabolic engineering
    * Dimerization of protein-protein interaction using small molcules
* Future improvements

# Background

## Scientific motivation


### "What I cannot create, I do not understand.”


#### Molecular recognition between small molecules and proteins

## Practical applications

* Diagnostics
    * [Point of care monitoring of compounds e.g phenylalanine](https://science.sciencemag.org/content/361/6407/1122)
        * Yu et al. Science 2018

* Metabolic Engineering
    * [Optimization of strains for metabolic engineering](https://www.pnas.org/content/113/9/2388.short) e.g. Ultra-high throughput screening of variants
        * Rogers et al. PNAS 2016

* Safety and convenience
    * [In vivo real time biosensors in plants](https://elifesciences.org/articles/10606) e.g. sensing drugs in real time (digoxigenin)
        * Feng et al. eLife 2015
        * Bick, Greisen et al. eLife 2017

# Ligand binding actually not a solved problem

![Back to square one...](img/PNAS_not_solved_problem.png)
<br>
**Main focus enzyme design but catalytic rates poor and substrate binding and orientation of side chains often wrong.... back to binding**

### Ligand design maybe harder than enzyme design
* Resolution of high-throughput experiment ~ uM
    * Enzyme assay can detect low amount of turn over
* In yeast and phage display 
    * Linker chemistry is a big perturbation

![MPA](img/mpa.png)
<br>
**Mycophenolic acid**

![MPA](img/MPA-PEG7-Biotin.png)
<br>
**PEG7 linker - PEG versus GS linker. Chemistry will affect binding of ligand for experiment**

# Steady progress but still room for improvement

## Seminal works within computational ligand design

* ["Computational design of ligand-binding proteins with high affinity and selectivity" - Tinberg et al. Nature 2013](https://www.nature.com/articles/nature12443)
* ["Engineering an allosteric transcription factor to respond to new ligands" - Taylor et al. Nature Methods 2016](https://www.nature.com/articles/nmeth.3696)
* ["Computational design of environmental sensors for the potent opioid fentanyl" - Bick, Greisen et al. eLife 2017](https://elifesciences.org/articles/28909)
* ["De novo design of a fluorescence-activating β-barrel" - Dou et al. Science 2018](https://www.nature.com/articles/s41586-018-0509-0)
* "Multi-input chemical control for graded and proportional programming of cellular processes" - Foight, Wang, Wei, Greisen et al. *submitted*

# Computational design of Fentanyl binder

## Fentanyl 
* Opioid targeting the μ-receptor
    * 100 x stronger then morphine
    * Elephant morphine
* Half of all reported opioid-related deaths
* Russian anti-terror molecule
* Chemical composition
    * 2 hydrogen bond acceptors
    * 6 rotatable bonds
    * 1 postive charge
    * 25 heavy atoms

In [1]:
fen = "CCC(=O)N(C1CC[NH+](CC1)CCC2=CC=CC=C2)C3=CC=CC=C3"
dig = "C[C@]12CC[C@@H](C[C@H]1CC[C@@H]3[C@@H]2C[C@H]([C@]4([C@@]3(CC[C@@H]4C5=CC(=O)OC5)O)C)O)O"

# SMILES from pubchem except CVX build from RVX
ligands = {'FEN' : fen,
           'DIG' : dig}

In [2]:
import shutil,subprocess,os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms as rdmt
import numpy as np
# from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolDrawing, DrawingOptions
from rdkit.Geometry import rdGeometry as geom
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact, interactive, fixed
from ipywidgets import interact, widgets
from IPython.display import display
import py3Dmol

In [3]:
def drawit(m,p,confId=-1):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    return p.show()

In [4]:
def generate_molecule(name,smiles):
    """
    Generate the 3D molecular structure based on input SMILES
    ----------
    name : name of molecule
    smiles: SMILES of molecule
    Returns
    ----------
    Mol 
    
    """
    LIGAND_NAME = name
    m = Chem.MolFromSmiles(smiles)
    m_h = Chem.AddHs(m)
    # Embeed the geometry
    AllChem.EmbedMolecule(m_h, AllChem.ETKDG())
    # Setting name of molecule
    m_h.SetProp("_Name",LIGAND_NAME)
    return m_h

In [5]:
def get_conformers(mol,nr=500,rmsthreshold=0.1):
    """
    Generate 3D conformers of molecule using CSD-method
    ----------
    mol : RKdit molecule
    nr : integer, number of conformers to be generate
    rmsthreshold : float, prune conformers that are less rms away from another conf
    Returns
    ----------
    List of new conformation IDs
    """
    # Generate conformers on the CSD-method
    return AllChem.EmbedMultipleConfs(mol, numConfs=nr,useBasicKnowledge=True,\
                                      pruneRmsThresh=rmsthreshold,useExpTorsionAnglePrefs=True)

In [6]:
mols = {}
for name in ligands.keys():
    mols[name] = generate_molecule(name,ligands[name])

In [7]:
for i in ligands.keys():
    cids = get_conformers(mols[i], 50,0.1)
    # Do a short minimization and compute the RMSD
    for cid in cids:
        _ = AllChem.MMFFOptimizeMolecule(mols[i], confId=cid)
    rmslist = []
    AllChem.AlignMolConformers(mols[i], RMSlist=rmslist)

In [8]:
molecule_widget = widgets.Dropdown(
    options=list(mols.keys()),
    value=list(mols.keys())[0],
    description='Molecule:',
    disabled=False,
)

In [9]:
molconf_widget = widgets.Label(
    value = 'Number of conformers: '+str(mols[molecule_widget.value].GetNumConformers())
)

In [10]:
def select_molecule(molecule):
    new_i = widgets.interactive(print_city, country=countryW, city=geoWs[country['new']])
    i.children = new_i.children
    
def on_change(change):
    from IPython.display import clear_output
    clear_output()
    molconf_widget.value = 'Number of conformers: '+str(mols[change['new']].GetNumConformers())
    display(container)
    interact(drawit,m=fixed(mols[change['new']]),p=fixed(p),confId=(0,mols[change['new']].GetNumConformers()-1));

In [11]:
container = widgets.HBox([molecule_widget, molconf_widget])
display(container)
# now construct the view and interactive widget:
p = py3Dmol.view(width=600,height=400)
# this is the widget that needs to tricker events
molecule_widget.observe(on_change, names='value')
interact(drawit,m=fixed(mols[molecule_widget.value]),p=fixed(p),confId=(0,mols[molecule_widget.value].GetNumConformers()-1));

interactive(children=(IntSlider(value=0, description='confId', max=49), Output()), _dom_classes=('widget-inter…

In [12]:
def generate_rosetta_input(mol,name,charge=0):
    try:
        os.mkdir(name)
    except:
        print("Directory already there!")
    os.chdir(name)
    w = Chem.SDWriter(name+'.sdf')
    for i in mol.GetConformers():
        w.write(mol,confId=i.GetId())
    w.close()
    parm_exe="python2.7 /Users/pgreisen/Programs/Rosetta/Rosetta/main/source/scripts/python/public/molfile_to_params.py "
    exe=parm_exe+" "+name+".sdf -n "+name+" --conformers-in-one-file --recharge="+str(charge)+" -c "
    subprocess.Popen(exe,shell=True)
    print(exe)
    os.chdir('../')

python ~/Rosetta/main/source/src/python/apps/public/molfile_to_params.py Fentanyl_cambridge.mol -n CFN -c --recharge=1

# Computational Protocol for design of small molecule binder

<center>
<img src="img/protocol.jpg" width="400">
</center>

In [13]:
computational_design_fen49 = "/Users/pgreisen/Documents/Presentations/pdbs/FEN49.pdb"
pdbdata=open(computational_design_fen49, 'r').read()
view=py3Dmol.view()

fen49 = {'chain':'A'}
fen = {'chain':'X'}

view.addModel(pdbdata,'pdb')
view.setStyle(fen49,{'cartoon': {'color':'white'}})

design_residues_nr = ['8','34','64','66','62','68','77']
for i in design_residues_nr:    
    mod_res = {'resi': i, 'chain': 'A'}
    col = 'green'
    c_col = col + 'Carbon'
    view.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
view.addStyle(fen, {'stick':{'colorscheme':'cyan'+ 'Carbon', 'radius': 0.2}})
view.addStyle(fen, {'sphere':{'color':'gray', 'opacity': 0.5}})          
view.zoomTo(fen,0.5)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
wt_2qz3 = "/Users/pgreisen/Documents/Presentations/pdbs/2qz3_chainA.pdb"
pdbdata=open(wt_2qz3, 'r').read()
view=py3Dmol.view()
view.addModel(pdbdata,'pdb')
wt_prt = {'chain':'A'}
xyp = {'resn':'XYP'}

view.setStyle(wt_prt,{'cartoon': {'color':'white'}})
design_residues_nr = ['9','35','65','67','63','69','78']

for i in design_residues_nr:    
    mod_res = {'resi': i, 'chain': 'A'}
    col = 'green'
    c_col = col + 'Carbon'
    view.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
view.addStyle(xyp, {'stick':{'colorscheme':'cyan'+ 'Carbon', 'radius': 0.2}})
view.addStyle(xyp, {'sphere':{'color':'gray', 'opacity': 0.5}})          

view.zoomTo({'resn': 'XYP'},0.5)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<center>

<img src="img/binding_site.jpg" width="400">

</center>

<center>

<img src="img/binding_site_zoom.png" width="600">

</center>

# Biophysical characterization of fentanyl binder

<center>

<img src="img/elife-28909-fig2-v2.jpg" width="400">

</center>

# Applications

## Degron system for generating biosensors

<center>
    <img src="img/degron_wandless.png" width="400">
</center>

**Bonger et al. NCB 2012**

<center>

<img src="img/elife-28909-fig4-v2.jpg" width="400">

</center>

# Specificty of computational designs
## Non-specific binding

## Works addressing ligand specificity

* ["Unintended specificity of an engineered ligand-binding protein facilitated by unpredicted plasticity of the protein fold" - Day et al. PEDS 2018](https://academic.oup.com/peds/article-abstract/31/10/375/5253233)
* ["Sampling and energy evaluation challenges in ligand binding protein design" - Dou et al. Protein Science 2017 ](https://onlinelibrary.wiley.com/doi/full/10.1002/pro.3317)

## Problems with Rosetta energy function
* Solvation
* Multiple states
* Movement of backbone
    * Secondary structure design
* Eletrostatics important for small molecules
* Ligand conformations
* Sampling
* Negative design
    * Hard to choose which ligands

# Hydrophobic ligands
## vitamin D3
## THC

In [15]:
def generate_molecule(name,smiles):
    """
    Generate the 3D molecular structure based on input SMILES
    ----------
    name : name of molecule
    smiles: SMILES of molecule
    Returns
    ----------
    Mol 
    
    """
    LIGAND_NAME = name
    m = Chem.MolFromSmiles(smiles)
    m_h = Chem.AddHs(m)
    # Embeed the geometry
    AllChem.EmbedMolecule(m_h, AllChem.ETKDG())
    # Setting name of molecule
    m_h.SetProp("_Name",LIGAND_NAME)
    return m_h

def get_conformers(mol,nr=500,rmsthreshold=0.1):
    """
    Generate 3D conformers of molecule using CSD-method
    ----------
    mol : RKdit molecule
    nr : integer, number of conformers to be generate
    rmsthreshold : float, prune conformers that are less rms away from another conf
    Returns
    ----------
    List of new conformation IDs
    """
    # Generate conformers on the CSD-method
    return AllChem.EmbedMultipleConfs(mol, numConfs=nr,useBasicKnowledge=True,\
                                      pruneRmsThresh=rmsthreshold,useExpTorsionAnglePrefs=True)




def select_molecule(molecule):
    new_i = widgets.interactive(print_city, country=countryW, city=geoWs[country['new']])
    i.children = new_i.children
    
def on_change(change):
    from IPython.display import clear_output
    clear_output()
    molconf_widget2.value = 'Number of conformers: '+str(newmols[change['new']].GetNumConformers())
    display(container2)
    interact(drawit,m=fixed(newmols[change['new']]),p=fixed(p),confId=(0,newmols[change['new']].GetNumConformers()-1));



thc = "CCCCCC1=CC2=C(C3C=C(CCC3C(O2)(C)C)C)C(=C1)O"
vit = "CC(CCCC(C)(C)O)C1CCC2C1(CCCC2=CC=C3CC(CCC3=C)O)C"


# SMILES from pubchem
newligands = {'VIT' : vit,
              'THC' : thc}

newmols = {}
for name in newligands.keys():
    newmols[name] = generate_molecule( name, newligands[name])

for i in newligands.keys():
    newcids = get_conformers(newmols[i], 50,0.1)
# Do a short minimization and compute the RMSD
for cid in newcids:
    _ = AllChem.MMFFOptimizeMolecule( newmols[i], confId=cid)
newrmslist = []
AllChem.AlignMolConformers( newmols[i], RMSlist=newrmslist)

molecule_widget2 = widgets.Dropdown(
    options=list(newmols.keys()),
    value=list(newmols.keys())[0],
    description='Molecule:',
    disabled=False,
)
molconf_widget2 = widgets.Label(
    value = 'Number of conformers: '+str(newmols[molecule_widget2.value].GetNumConformers())
)


container2 = widgets.HBox([molecule_widget2, molconf_widget2])
display(container2)
# now construct the view and interactive widget:
p = py3Dmol.view(width=600,height=400)
# this is the widget that needs to tricker events
molecule_widget2.observe(on_change, names='value')
interact(drawit,m=fixed(newmols[molecule_widget2.value]),p=fixed(p),confId=(0,newmols[molecule_widget2.value].GetNumConformers()-1));

interactive(children=(IntSlider(value=0, description='confId', max=49), Output()), _dom_classes=('widget-inter…

## Docking of vitamin D3 into 3 variants

<center>
    <img src="img/docking_vitaminD3.png" width="400">
</center>

**Day et al. PEDS 2018**

# Applications of biosensors

## Dimerization of protein-protein interaction using small molcules
<center>
    <img src="img/danoprovir.png" width="400">
</center>

**Foight, Wang, Wei, Greisen** *submitted to Nature Biotechnology*

## Biosensor in yeast
<center>
    <img src="img/yeast_sensorsystem.png" width="400">
</center>

**Ben Jester** *unpublished data*

# Perspective and conclusion

* Ligand binding is hard
    * Specificity
    * Affinity
* Energy function improvements
    * Electrostatics
    * Ligand conformations
* Increase of data
    * Crystal structures
    * Sequencing

# Acknowledgement 

* Sergey Ovchinnikov
* Austin Day
* Matt Bick
* Jiayi Dou
* Zhizhi Wang
* Yi Sheng
* Glenna Foight
* Ben Jester
* Christy Tinberg
* Sagar Khare
* David Baker

